In [2]:
import re
import pandas as pd
import tqdm, time
file_dir = '~/Documents/Materials/iron/'

#### 1. generate a txt file with format chrN:12345678-12345678 using chromosome and hg19_position from file all_variants_and_proxies.csv

In [31]:
df = pd.read_csv(file_dir+'all_variants_and_proxies.csv')
# df['format'] = 'chr'+df.chromosome.astype(str)+':'+df.hg19_position.astype(str)+'-'+df.hg19_position.astype(str)
# df['format'].to_csv('~/Documents/Projects/iron/variants_hg19.txt', index=False) 

In [52]:
df[df.hg19_position == 26104632]

,chromosome,hg19_position,rsid,ea,nea,eaf,hg38_position,finn_format,finn_af,finn_rsid,finn_ref,finn_alt,match,finn_hg38,sandbox_format
946,6,26104632,rs198851,T,G,0.15148,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G
947,6,26104632,rs198851,G,T,0.84852,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G


In [53]:
df[df.sandbox_format.isin(['chr6_26090957_A_T', 'chr6_26092913_G_A', 'chr6_26104404_T_G'])]

,chromosome,hg19_position,rsid,ea,nea,eaf,hg38_position,finn_format,finn_af,finn_rsid,finn_ref,finn_alt,match,finn_hg38,sandbox_format
939,6,26091185,rs1800730,T,A,0.016247,26090957,6:26090957:A:T (rs1800730),0.021,rs1800730,A,T,True,26090957.0,chr6_26090957_A_T
942,6,26093141,rs1800562,A,G,0.077196,26092913,6:26092913:G:A (rs1800562),0.037,rs1800562,G,A,True,26092913.0,chr6_26092913_G_A
943,6,26093141,rs1800562,G,A,0.922804,26092913,6:26092913:G:A (rs1800562),0.037,rs1800562,G,A,True,26092913.0,chr6_26092913_G_A
946,6,26104632,rs198851,T,G,0.151480,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G
947,6,26104632,rs198851,G,T,0.848520,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G


#### 2. convert hg19 coordinates to hg38 coordinates using liftover tool: 
https://genome.ucsc.edu/cgi-bin/hgLiftOver

#### 3. rename the file downloaded from the tool as variants_hg19.bed and then add it to all_variants_and_proxies.csv

In [7]:
df0 = pd.read_csv(file_dir+'variants_hg38.bed', header=None) 
df['hg38_position'] = df0[0].str.extract('\:(\d+)\-') 
df.to_csv(file_dir+'all_variants_and_proxies.csv', index=False)

#### 4. scrape off head and AF from https://results.finngen.fi/ using rsid
backend of finngen dashboard: https://github.com/FINNGEN/saige-pipelines/blob/master/wdl/gwas/saige.json

e.g. Given rs1961437,

```head = '22:37114993:A:G (rs1961437)'``` 

```str_af = 'AF 9.1e-1(ranges from 9.1e-1 to 9.1e-1 across all phenotypes)'```

```af = 0.91```

In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
browser = webdriver.Chrome(Service('/Users/feiwang/Documents/Projects/chromedriver'))
browser.get('https://results.finngen.fi/')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


In [160]:
finngen_format_list, finngen_af_list = [], []

for i in tqdm.tqdm(df.rsid.tolist()):
#     browser.get('https://results.finngen.fi/variant/'+1:22227885-T-C)
    # there are 9 SNPs without rsid in the file
    if type(i) == float:
        finngen_format_list.append('id isna')
        finngen_eaf_list.append('id isna')
    else:
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(i)
        browser.find_element(By.XPATH, '//*[@id="navbar-searchbox-input"]').send_keys(Keys.RETURN)
        time.sleep(5)
        try:
            head = browser.find_element(By.XPATH, '/html/body/div/div[1]/div/h1').text
        except:
    #         err = browser.find_element_by_xpath('//*[@id="reactEntry"]/div/div/div/p[1]').text
            head = 'id notfound'
        if head == 'id notfound':
            eaf = 'id notfound'
        else:
            try:
                str_af = browser.find_element(By.XPATH, '//*[@id="maf-range"]/p').text
                af = float(re.findall('AF ([\d|\.|e|\-]+)\(ranges', str_eaf)[0])
            except:
                af = 'eaf notfound'

        finngen_format_list.append(head)
        finngen_af_list.append(af)
        time.sleep(1)

  0%|          | 0/1039 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
100%|██████████| 1039/1039 [1:50:57<00:00

In [169]:
df['finn_format'] = finngen_format_list
df['finn_af'] = finngen_af_list

In [198]:
df['finn_rsid'] = df.finn_format.str.extract(' \((rs\d+)\)')
df['finn_ref'] = df.finn_format.str.extract(':(\w+):\w+ \(')
df['finn_alt'] = df.finn_format.str.extract(':\w+:(\w+) \(')

In [199]:
df['match'] = df.finn_rsid == df.rsid

In [204]:
for i, row in df[(df.match == False)&(df.finn_rsid.isna())&(df.finn_format != 'id notfound')].iterrows():
    id_list = re.findall('(rs\d+)',row.finn_format)
    if row.rsid in id_list:
        df.loc[i,'finn_rsid'] = row.rsid
        df.loc[i,'match'] = True

In [245]:
print(len(df[df.match == False]),'cannot be found unfortunately')
print(len(df)-len(df[df.match == False]),'were found successfully')

514 cannot be found unfortunately
2672 were found successfully


#### 5. create a list of format that can be recognized in finngen sandbox

In [241]:
df['finn_hg38'] = df.finn_format.str.extract('\d{1,2}\:(\d+)\:\w+\:')
df['sandbox_format'] = 'chr'+df.chromosome.astype(str)+'_'+df.finn_hg38+'_'+df.finn_ref+'_'+df.finn_alt
df.to_csv(file_dir+'all_variants_and_proxies.csv', index=False)

#### 6. obtain phecode_map from PheWAS package in R
See phecode_map.R for detail
#### 7. label a PheCode if it is sex-specified
Download sex-specified PheCode dictionary from https://phewascatalog.org/files/phecode_definitions1.2.csv.zip

In [411]:
phecode_map = pd.read_csv(file_dir+'phecode_map.csv') 
phecode_def = pd.read_csv(file_dir+'phecode_definitions1.2.csv') 
phecode_list = phecode_map.phecode.unique().tolist()
phecode_dict = pd.DataFrame({'phecode':phecode_list})
phecode_dict = phecode_dict.merge(phecode_def[['phecode','sex']], 'left')

In [416]:
phecode_sex = phecode_dict[(~phecode_dict.sex.isna())&(phecode_dict.sex != 'Both')]
# in total 166 sex-specified phecodes
phecode_sex.to_csv(file_dir+'phecode_sex.csv', index=None)

#### 8. upload the files to green bucket: 
https://console.cloud.google.com/storage/browser/fg-production-sandbox-6_green

gsutil cp ~/Documents/Materials/iron/all_variants_and_proxies.csv gs://fg-production-sandbox-6_green/FeiyiWang

gsutil cp ~/Documents/Materials/iron/phecode_map.csv gs://fg-production-sandbox-6_green/FeiyiWang

gsutil cp ~/Documents/Materials/iron/phecode_sex.csv gs://fg-production-sandbox-6_green/FeiyiWang

#### 9. rerun the process for the final version:

In [5]:
df = pd.read_csv(file_dir+'all_variants_and_proxies.csv')
df = df[df.match == True]
df_final = pd.read_csv(file_dir+'all_variants_and_proxies_finn_af_final.csv')
df_final['index'] = df_final.index
# add sandbox format to df_final
df_final = df_final.merge(df[['rsid','sandbox_format']].drop_duplicates(),'left')
# save the data
df_final.to_csv(file_dir+'all_variants_and_proxies_finn_af_final.csv', index=None)
# upload the data to the green bucket

gsutil cp ~/Documents/Materials/iron/all_variants_and_proxies_finn_af_final.csv gs://fg-production-sandbox-6_green/FeiyiWang

#### 10. download summary statistics
download the final results from green box to local machine for 2SMR analysis

gsutil cp gs://given_link/results.csv ~/Documents/Materials/iron/

chr6_26090957_A_T, chr6_26092913_G_A, chr6_26104404_T_G

- '401': 'I9_HYPTENS',
- '280': 'D3_ANAEMIA_IRONDEF',
- '285': 'D3_ANAEMIANAS',
- '275': 'E4_MINERAL_MET'

In [45]:
phecode_def = pd.read_csv(file_dir+'phecode_definitions1.2.csv') 

In [49]:
phecode_def[phecode_def.phecode == 275.]

,phecode,phenotype,phecode_exclude_range,sex,rollup,leaf,category_number,category
356,275.0,Disorders of mineral metabolism,275-275.99,NaN,1,0,3,endocrine/metabolic


In [43]:
results = pd.read_csv(file_dir+'results.csv', dtype={5:'str'})
results = results.rename(columns={'Unnamed: 0':'snp'})

In [30]:
results[results.snp.str.startswith('chr6_2609')].snp.unique()

array(['chr6_26090951_C_G', 'chr6_26092913_G_A', 'chr6_26098246_T_C',
       'chr6_26090957_A_T'], dtype=object)